In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')
from datetime import datetime


In [3]:
merged_click_customer= pd.read_parquet('merged_click_customer.parquet', engine='pyarrow')

ArrowMemoryError: realloc of size 268435456 failed

In [ ]:
churn_rate =  merged_click_customer[['customer_id','session_id','event_name','event_time','total_amount','first_join_date']]

# 'event_time'과 'first_join_date' 열을 datetime 형태로 변환
churn_rate['event_time'] = pd.to_datetime(churn_rate['event_time'])
churn_rate['first_join_date'] = pd.to_datetime(churn_rate['first_join_date'])

In [ ]:
start_date = '2016-01-01'
end_date = '2016-12-31'
churn_16 = churn_rate.query(f'event_time >= "{start_date}" & event_time <= "{end_date}"')

In [ ]:
# 'BOOKING' 이벤트만 필터링
booking_df = churn_16[churn_16['event_name'] == 'BOOKING']
purchase_counts = booking_df.groupby(['customer_id', 'event_time']).size().reset_index(name='purchase_count')
purchase_counts['next_event'] = purchase_counts['event_time'].shift(-1)
purchase_counts['date_diff'] = (purchase_counts['next_event'] - purchase_counts['event_time']).dt.days
purchase_counts['month'] = purchase_counts['event_time'].dt.strftime('%Y-%m')

purchase_counts = purchase_counts.query('date_diff >= 0')

In [ ]:
# 리텐션 조건
# purchase_counts['repurchase'] = np.where(purchase_counts['date_diff'] <= 15, 1, 0)

In [ ]:
total_customers = purchase_counts.drop_duplicates('customer_id')['customer_id'].count()
repurchase = purchase_counts.groupby('month')['customer_id'].count()

repurchase_rate = (repurchase / total_customers).dropna()
repurchase_rate.plot(kind='bar', figsize=(12, 6))

plt.title('Repurchase Rate Over Time')
plt.xlabel('Period')
plt.ylabel('Repurchase Rate')
plt.xticks(rotation=45)
plt.show()